In [1]:
!pip install telethon
!pip install unidecode
!pip install pillow

from telethon.sync import TelegramClient
from telethon.tl.types import InputMessagesFilterPhotos, InputMessagesFilterDocument
from telethon.tl.functions.messages import GetHistoryRequest

import os
import json
from datetime import datetime
from unidecode import unidecode
from PIL import Image
import io


  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 722.0/722.0 kB 6.9 MB/s eta 0:00:00
  Created wheel for pyaes: filename=pyaes-1.6.1-py3-none-any.whl size=26347 sha256=3d718eea5ba582797b6e363c57eee042f653eb4de27a1eab9b73df3a6fc788e3
  Stored in directory: /root/.cache/pip/wheels/4e/52/33/010d0843550bffb6a591b11629070ae140c0ad4f53e68a3bd3
Successfully built pyaes
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.8/235.8 kB 7.4 MB/s eta 0:00:00


In [ ]:
import os
from dotenv import load_dotenv

load_dotenv()  # Load variables from .env file

DB_NAME = os.getenv("DB_NAME")
DB_USER = os.getenv("DB_USER")

In [ ]:
# Set up Telegram API credentials
api_id = os.getenv("API_ID")  # e.g., 123456
api_hash = os.getenv("API_HASH")  # e.g., 'abcdef1234567890abcdef1234567890'

# Start a session
client = TelegramClient('amharic_scraper_session', api_id, api_hash)
await client.start()


Please enter your phone (or bot token): +251926463561
Please enter the code you received: 81028
Please enter your password: ··········
Signed in successfully as Bereket Zewde; remember to not break the ToS or you will risk an account ban!


In [ ]:
# List  target channels here 
channel_usernames = [
    '@Ethiopiaonlin19',
    '@ethioomart2',
]


In [ ]:
from telethon.tl.functions.messages import GetHistoryRequest
from telethon.tl.types import InputPeerEmpty

async def fetch_channel_messages(channel_username, limit=100):
    messages_data = []

    entity = await client.get_entity(channel_username)

    history = await client(GetHistoryRequest(
        peer=entity,
        offset_id=0,
        offset_date=None,
        add_offset=0,
        limit=limit,
        max_id=0,
        min_id=0,
        hash=0
       
    ))

    for msg in history.messages:
        if msg.message:
            data = {
                'channel': channel_username,
                'message_id': msg.id,
                'sender_id': msg.from_id.user_id if msg.from_id else None,
                'timestamp': msg.date.isoformat(),
                'text': msg.message,
                'has_media': msg.media is not None
            }

            if msg.media:  # download image or media 
                file_path = f'/content/{channel_username}_{msg.id}.jpg'
                await client.download_media(msg, file_path)
                data['image_path'] = file_path

            messages_data.append(data)

    return messages_data


In [5]:
all_data = []

for channel in channel_usernames:
    print(f"Fetching from {channel}")
    channel_data = await fetch_channel_messages(channel, limit=100)
    all_data.extend(channel_data)


Fetching from @Ethiopiaonlin19
Fetching from @ethioomart2


In [ ]:
print(all_data)

[{'channel': '@Ethiopiaonlin19', 'message_id': 48468, 'sender_id': 6210545395, 'timestamp': '2025-06-21T09:20:13+00:00', 'text': 'ለሽያጭ የወጣ አፓርታማ \n\n👉 አያት 49 አካባቢ ምቹ የሆነ አካባቢ\n👉 1ኛ ፎቅ ላይ ባለ 3 መኝታ ቤት\n👉 ስፋት 162 ካሬ የውስጥ 111 ካሬ\n👉 ዶክመንት ዲጂታል ካርታ የያዙ \n👉 የባንክ ዕዳ 7 ሚሊዮን የሚመቻች\n👉 ጠቅላላ ዋጋ 11,2 ሚሊዮን\n👉 ከዕዳ ውጪ ለጁ 4,2 ሚሊዮን \n\nኮሚሽን 2% ይከፍላሉ።\n\n☎️ 09 23 25 02 13\n☎️ 07 16 19 19 89\n( ስልክ ካልተነሳ text ያድርጉ )', 'has_media': True, 'image_path': '/content/@Ethiopiaonlin19_48468.jpg'}, {'channel': '@Ethiopiaonlin19', 'message_id': 48461, 'sender_id': 6210545395, 'timestamp': '2025-06-21T08:30:52+00:00', 'text': 'ሀብታሙ ጋር ተፈትሾ ምንም ያልተገኝበት ንፁ መኪና \n\nModel Suzuki Celerio\nYear 2021\nPlate~Code 2 C22***\nTransmission:Automatic\nMilage~40,000 km\nEngine~1.0 L Fuel~Benzine\nCondition:Almost New\n800,000 ብር ባንክ ይመቻቻል \nCash 1,300,000 birr \nTotal price 2,100,000 birr\n\nኮሚሽን 2% ይከፍላሉ።\n\n☎️ 09 23 25 02 13\n☎️ 07 16 19 19 89\n( ስልክ ካልተነሳ text ያድርጉ )', 'has_media': True, 'image_path': '/content/@Ethiopiaonlin19

In [6]:
import re

def clean_amharic_text(text):
    # Remove emojis, links, etc.
    text = re.sub(r"http\S+|www\S+|https\S+", '', text)
    text = re.sub(r"[\U0001F600-\U0001F64F]", '', text)
    text = re.sub(r"[@#]\S+", '', text)
    text = re.sub(r"[^አ-ኯነ-኷ሀ-ሯቀ-ቧቨ-ቯተ-ቿኀ-ኋአ-ኧመ-፼ 0-9a-zA-Z]", ' ', text)
    text = re.sub(r"\s+", ' ', text).strip()
    return text

# Apply preprocessing
for record in all_data:
    if 'text' in record:
        record['clean_text'] = clean_amharic_text(record['text'])

# Save preprocessed version
preprocessed_path = 'telegram_ecommerce_data_cleaned.json'
with open(preprocessed_path, 'w', encoding='utf-8') as f:
    json.dump(all_data, f, ensure_ascii=False, indent=2)

print(f'Cleaned data saved to {preprocessed_path}')


Cleaned data saved to telegram_ecommerce_data_cleaned.json
